In [1]:
import minsearch
import pandas as pd
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
df = pd.read_csv('../data/krakow_pois_for_rag.csv')

text_columns = ['name','amenity','leisure','natural','tourism','historic','wiki_summary_en']

In [3]:
[col for col in df.columns if col =='poi_category']

[]

In [3]:
documents = df.to_dict(orient='records')

In [4]:
index = minsearch.Index(
    text_fields=text_columns,keyword_fields=['id'])

In [5]:
index.fit(documents)

In [6]:
query = 'what are some historical places to visit in krakow?'
results = index.search(query)

In [7]:
openai_client = OpenAI(api_key=OPENAI_API_KEY)


In [8]:
def search(query,filter_dict={},num_results=10,boost={}):

    results = index.search(
        query=query,
        filter_dict=filter_dict,
        boost_dict=boost,
        num_results=num_results
    )

    return results

In [9]:
entry_template = """

phone : {phone}
cemetery : {cemetery}
emergency : {emergency}
opening_hours : {opening_hours}
website : {website}
pets_allowed : {pets_allowed}
geometry : {geometry}
historic : {historic}
wiki_summary_en : {wiki_summary_en}
postal_code : {postal_code}
toilets : {toilets}
natural : {natural}
description : {description}
visiting_time : {visiting_time}
leisure : {leisure}
tourism : {tourism}
public_transport : {public_transport}
brand : {brand}
alt_name : {alt_name}
amenity : {amenity}
reservation : {reservation}
attraction : {attraction}
highchair : {highchair}
parking : {parking}
swimming_pool : {swimming_pool}
contact_phone : {contact_phone}
community_centre : {community_centre}
addr_street : {addr_street}
contact_twitter : {contact_twitter}
social_facility : {social_facility}
contact_facebook : {contact_facebook}
zoo : {zoo}
email : {email}
wheelchair : {wheelchair}
cuisine : {cuisine}
contact_website : {contact_website}
internet_access : {internet_access}
opening_hours_reception : {opening_hours_reception}
guest_house : {guest_house}
addr_city : {addr_city}
contact_instagram : {contact_instagram}
image : {image}
location : {location}
outdoor_seating : {outdoor_seating}
museum : {museum}
takeaway : {takeaway}
smoking : {smoking}
name : {name}
id : {id} """.strip()

In [ ]:
def build_prompt(query, search_results):
    prompt_template = """
You're a Krakow travel assistant (guide). Answer the QUESTION based on the CONTEXT.
Data from Context is from Krakow points of interest (POI) database. 
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [11]:
def llm(prompt):
    response = openai_client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [12]:
query = 'plan me a day in Krakow?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [13]:
answear = rag(query)

In [14]:
print(answear)

Here's a suggested day plan in Krakow:

**10:00 AM** – Start your day with a stroll through **Planty Park**. This scenic park encircles the historic Old Town and offers beautiful gardens, statues, and fountains.

**11:00 AM** – Visit the **National Museum in Kraków** located on Aleja 3 Maja. This museum is the largest in Poland, featuring diverse art collections. (Opening hours: Mo-Fr 10:00-22:00; Sa-Su 11:00-22:00).

**1:00 PM** – Enjoy lunch at **Me Thai**, a restaurant serving Asian and Thai cuisine, located on Kazimierza Brodzińskiego.

**2:30 PM** – After lunch, head to **MOCAK (Museum of Contemporary Art in Kraków)**. This museum focuses on contemporary art and has a library and café as well. It’s a great spot to explore modern artistic expressions.

**4:00 PM** – Check out the **Mikołaj Zyblikiewicz monument** near All Saints' Square. It's a historical memorial worth visiting during your walk.

**5:00 PM** – If time permits, visit the **Benedictine Abbey in Tyniec**. It’s slight